In [88]:
#NETFLIX REAL 50.000.000 usuaris and 100.000 items
%autosave 150
%matplotlib inline
import pandas as pd
import numpy as np
import math
import matplotlib.pylab as plt

# Load Data set
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('./data/ml-100k/u.user', sep='|', names=u_cols)

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('./data/ml-100k/u.data', sep='\t', names=r_cols)

# the movies file contains columns indicating the movie's genres
# let's only load the first three columns of the file with usecols
m_cols = ['movie_id', 'title', 'release_date']
movies = pd.read_csv('./data/ml-100k/u.item', sep='|', names=m_cols, usecols=range(3), encoding='latin-1')

# Construcció del DataFrame
data = pd.merge(pd.merge(ratings, users), movies)
data = data[['user_id','title', 'movie_id','rating','release_date','sex','age']]


print("La BD has "+ str(data.shape[0]) +" ratings")
print("La BD has ", data.user_id.nunique()," users")
print("La BD has ", data.movie_id.nunique(), " movies")
data.head()

Autosaving every 150 seconds
La BD has 100000 ratings
La BD has  943  users
La BD has  1682  movies


,user_id,title,movie_id,rating,release_date,sex,age
0,196,Kolya (1996),242,3,24-Jan-1997,M,49
1,305,Kolya (1996),242,5,24-Jan-1997,M,23
2,6,Kolya (1996),242,4,24-Jan-1997,M,42
3,234,Kolya (1996),242,4,24-Jan-1997,M,60
4,63,Kolya (1996),242,3,24-Jan-1997,M,31


In [2]:
user_item = ratings.pivot_table(index=['movie_id'],columns=['user_id'],values='rating')#.reset_index(drop=True)
user_item.fillna( 0, inplace = True )
index = user_item.index
user_item.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
movie_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,4.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,4.0,...,2.0,3.0,4.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0
2,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,...,5.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
5,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation
movie_similarity = pairwise_distances( user_item.to_numpy(), metric="cosine" )
np.fill_diagonal( movie_similarity, 0 ) #Filling diagonals with 0s for future use when sorting is done
ratings_matrix = pd.DataFrame( movie_similarity )
ratings_matrix.index=index
ratings_matrix.columns = index.to_list()
ratings_matrix

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
movie_id,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.597618,0.669755,0.545062,0.713286,0.883656,0.379021,0.518886,0.503712,0.726065,...,0.964613,1.0,1.000000,1.000000,0.964613,1.0,1.0,1.0,0.952817,0.952817
2,0.597618,0.000000,0.726931,0.497429,0.681164,0.916437,0.616597,0.662998,0.744748,0.828918,...,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.0,0.921701,0.921701
3,0.669755,0.726931,0.000000,0.675134,0.787043,0.893278,0.627079,0.799206,0.726331,0.841896,...,1.000000,1.0,1.000000,1.000000,0.967708,1.0,1.0,1.0,1.000000,0.903125
4,0.545062,0.497429,0.675134,0.000000,0.665761,0.909692,0.510717,0.509764,0.580956,0.747439,...,1.000000,1.0,0.905978,0.905978,0.962391,1.0,1.0,1.0,0.943587,0.924782
5,0.713286,0.681164,0.787043,0.665761,0.000000,0.962701,0.665231,0.740839,0.727552,0.944547,...,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.000000,0.905789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,0.0,0.0,0.0,1.000000,1.000000
1679,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,0.0,0.0,0.0,1.000000,1.000000
1680,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,0.0,0.0,0.0,1.000000,1.000000


In [14]:
N = 10
e = data_test[:N][["user_id", "movie_id"]]
estimated_rate = []
for i,r in e.iterrows():
    filtered = user_item[user_item[r["user_id"]]>0][r["user_id"]]
    rated_movies = (filtered).index.to_list()
    weights = ratings_matrix.iloc[r["movie_id"], rated_movies].to_numpy()
    num = np.multiply(filtered,weights).sum()
    den = weights.sum()
    rate = num/den
    estimated_rate.append(rate)
estimated_rate

[3.6606915152326707,
 3.4027358802229184,
 3.229515678420483,
 3.7526781854163085,
 4.186573466329074,
 3.9942877648430732,
 4.328051598974591,
 3.6810382752155935,
 2.129115311591276,
 3.138719764125569]

In [22]:
user_mean = pd.DataFrame(data_train.groupby('user_id')["rating"].mean()).reset_index()
user_mean = user_mean.rename(columns={"rating": "mean_rating"})
data_train = pd.merge(data_train, user_mean, on="user_id")
data_train["normalized_rating"] = data_train["rating"] - data_train["mean_rating"]

,user_id,mean_rating
0,1,3.694836
1,2,3.685185
2,3,2.767442
3,4,4.238095
4,5,2.842857
...,...,...
938,939,4.285714
939,940,3.519481
940,941,4.052632
941,942,4.224138


# Item-Based Model

In [443]:
from sklearn.metrics import pairwise_distances
import pdb
from scipy.spatial.distance import cosine
import scipy.sparse as sparse

def custom_cosine(x,y):
    mask = (x !=0) & (y!=0)
    if any(mask):
        d = 1-cosine(x[mask], y[mask])
        return d if d>0 else 0
    else:
        return 0
    
class ItemBasedCF:
    def __init__(self, knei=None):
        self.knei = knei
        
    def fit(self, data_train, normalize=True):
        self.df = data_train
        
        if normalize:
            user_mean = pd.DataFrame(data_train.groupby('user_id')["rating"].mean()).reset_index()
            user_mean = user_mean.rename(columns={"rating": "mean_rating"})
            data_train = pd.merge(data_train, user_mean, on="user_id")
            data_train["normalized_rating"] = data_train["rating"] - data_train["mean_rating"] #Normalize
            fileterd_df = data_train[["user_id", "movie_id", "rating", "normalized_rating"]]
            self.user_item = fileterd_df.pivot_table(index=['movie_id'],columns=['user_id'],values='normalized_rating')#.reset_index(drop=True)
        else:
            fileterd_df = data_train[["user_id", "movie_id", "rating"]]
            self.user_item = fileterd_df.pivot_table(index=['movie_id'],columns=['user_id'],values='rating')#.reset_index(drop=True)
        self.user_item.fillna(0, inplace = True )
        index = self.user_item.index
        
        movie_similarity = pairwise_distances( self.user_item.to_numpy(), metric=custom_cosine)
        np.fill_diagonal(movie_similarity, 0 ) #Filling diagonals with 0s for future use when sorting is done
        self.sim_matrix = pd.DataFrame(movie_similarity)
        self.sim_matrix.index = index
        self.sim_matrix.columns = index.to_list()
        if normalize: self.user_item = fileterd_df.pivot_table(index=['movie_id'],columns=['user_id'],values='rating')
        self.sparse_user_item = sparse.csr_matrix(self.user_item.fillna(0).T)
        self.sim_matrix_np = movie_similarity
        
    def estimate_df(self, df):
        e = df[["user_id", "movie_id"]]
        estimated_rate = []
        for i,r in e.iterrows():
            rate = self.estimate(r["user_id"], r["movie_id"])
            estimated_rate.append(rate)
        return np.array(estimated_rate)
    
    def estimate(self, user_id, movie_id):
        filtered = self.user_item[self.user_item[user_id]>0][user_id]
        rated_movies = (filtered).index.to_list()

        if movie_id in self.sim_matrix.index:
            weights = self.sim_matrix.loc[movie_id, rated_movies].to_numpy()
            arg_sort = np.argsort(weights)[::-1]
        else:
            return filtered.mean()
        num = np.multiply(filtered.to_numpy()[arg_sort],weights[arg_sort])
        if self.knei:
            num = num[:self.knei].sum()
            den = weights[arg_sort][:self.knei].sum()
        else:
            num = num.sum()
            den = weights.sum()
        rate = num/den
        return rate
    
    def user_recomendation(self, user_index, selection = None):
        user_sparse_vector = self.sparse_user_item[user_index]
        rated_items = self.sparse_user_item[user_index].indices


        movies_to_rate_mask = np.ones(self.sim_matrix_np.shape[0], dtype=bool)
        movies_to_rate_mask[rated_items] = False
        if selection:
            i,= np.where(a)
            i = np.random.choice(i,selection,replace=False)
            movies_to_rate_mask[i]=False
        movies_to_rate = self.sim_matrix_np[:,movies_to_rate_mask]
        movies_to_rate = movies_to_rate[rated_items,:]

        if not self.knei:
            num = (user_sparse_vector.data).dot(movies_to_rate)
            den = movies_to_rate.sum(axis=0)
            ratings = np.divide(num, den)
        else:
            qty = movies_to_rate.shape[1]
            ratings = np.zeros(qty)
            for j in range(qty):
                weights = movies_to_rate[:,j]
                arg_sort = np.argsort(weights)[::-1]
                user = user_sparse_vector.data[arg_sort]
                weights = weights[arg_sort]
                ratings[j] = user.dot(weights)/weights.sum()
        index_movies, = np.where(movies_to_rate_mask)
        return [self.sim_matrix.columns[m_ix] for r, m_ix in sorted(zip(ratings, index_movies), reverse=True)]
        
    def all_useres_recomendation(self, N=10, selection=None):
        user_indexes = self.user_item.index
        recomendation = []
        for u_index in range(self.sparse_user_item.shape[0]):
            recomendation.append(self.user_recomendation(u_index, selection=selection)[:N])
        return np.array(recomendation)
    
def compute_rmse(y_pred, y_true):
    return np.sqrt(np.mean(np.power(y_pred - y_true, 2)))   
    
def compute_mae(y_pred, y_true):
    return np.mean(np.abs(y_pred - y_true))
      
def evaluate_mae_rmse(estimate_f,data_train,data_test):
    """ RMSE-based predictive performance evaluation with pandas. """
    ids_to_estimate = zip(data_test.user_id, data_test.movie_id)
    estimated = np.array([estimate_f(u,i) if u in data_train.user_id else 3 for (u,i) in ids_to_estimate ])
    real = data_test.rating.values
    return compute_rmse(estimated, real), compute_mae(estimated, real)


In [270]:
test_size = 0.1
def assign_to_set(df):
    sampled_ids = np.random.choice(df.index,
                                   size=np.int64(np.ceil(df.index.size * test_size)),
                                   replace=False)
    df.loc[sampled_ids, 'for_testing'] = True
    return df
df = data

#data#[data['movie_id']<100] # get only data from 100 movies
print("Total movies", len(df.movie_id.unique()))
movie_stats = df.groupby('movie_id').agg({'rating': [np.size, np.mean]})
min_20 = movie_stats['rating']['size'] >= 20
filtered_movies_id = movie_stats[min_20].index
print("Filtered movies ", len(filtered_movies_id.to_list()))
df = df[df.movie_id.isin(filtered_movies_id)]

df.loc[:,'for_testing'] = False
grouped = df.groupby('movie_id', group_keys=False).apply(assign_to_set)
data_train = df[grouped.for_testing == False]
data_test = df[grouped.for_testing == True]

print(data_train.shape )
print(data_test.shape )

print('Users:', df.user_id.nunique() )
print('Movies:',df.movie_id.nunique() )

Total movies 1682
Filtered movies  939


/Users/emiliotylson/.local/share/virtualenvs/RECOMENDERS-DgpJbO42/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/emiliotylson/.local/share/virtualenvs/RECOMENDERS-DgpJbO42/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


(85043, 8)
(9925, 8)
Users: 943
Movies: 939


In [271]:
%%time
reco = ItemBasedCF(knei=40)
reco.fit(data_train, normalize=False)
#reco.estimate(user_id=2,movie_id=1)

CPU times: user 17.9 s, sys: 52.2 ms, total: 17.9 s
Wall time: 18 s


In [272]:
%%time
rmse, mae = evaluate(reco.estimate, data_train, data_test)
print('MAE for Collaborative Recomender: %s' % mae)

MAE for Collaborative Recomender: 0.8012462140781225
CPU times: user 9.33 s, sys: 209 ms, total: 9.54 s
Wall time: 9.56 s


In [463]:
%%time
reco = ItemBasedCF(knei=40)
reco.fit(data_train, normalize=True)
#reco.estimate(user_id=2,movie_id=1)

CPU times: user 20.9 s, sys: 122 ms, total: 21 s
Wall time: 22.3 s


In [274]:
%%time
rmse, mae =  evaluate(reco.estimate, data_train, data_test)
print('MAE for Collaborative Recomender Normalized: %s' % mae)

MAE for Collaborative Recomender Normalized: 0.752923388619128
CPU times: user 9.24 s, sys: 98.3 ms, total: 9.34 s
Wall time: 9.36 s


In [438]:
%%time
reco = ItemBasedCF()
reco.fit(data_train, normalize=False)

CPU times: user 21.7 s, sys: 228 ms, total: 22 s
Wall time: 28.4 s


In [439]:
%%time
m = reco.all_useres_recomendation(N=10)
m

CPU times: user 6.72 s, sys: 450 ms, total: 7.17 s
Wall time: 3.78 s


In [444]:
%%time
reco = ItemBasedCF(knei=40)
reco.fit(data_train, normalize=False)

CPU times: user 18 s, sys: 64.2 ms, total: 18 s
Wall time: 18.1 s


In [445]:
%%time
m = reco.all_useres_recomendation(N=10)
m

CPU times: user 11.2 s, sys: 30.1 ms, total: 11.2 s
Wall time: 11.2 s


array([[1149, 1203,  962, ...,  460, 1022, 1008],
       [ 375, 1170, 1093, ...,  760, 1139,  320],
       [1121, 1203, 1063, ..., 1036, 1139, 1045],
       ...,
       [ 874,  937, 1126, ...,  320,  770,  601],
       [1121,  961, 1203, ..., 1208,  383, 1118],
       [1022,  335,  962, ..., 1142, 1008,   19]])